###여러 독립변수와 종속변수 사용

In [1]:
import tensorflow as tf
tf.__version__

'2.9.2'

###Raw Data Loading

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./admission.csv')
# df
# df.describe()
# df.info()
df.keys()

Index(['admit', 'gre', 'gpa', 'rank'], dtype='object')

In [4]:
train_df = df[['gre', 'gpa', 'admit']]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gre     400 non-null    int64  
 1   gpa     400 non-null    float64
 2   admit   400 non-null    int64  
dtypes: float64(1), int64(2)
memory usage: 9.5 KB


###Data PreProcessing

#####각 columns 분리

In [5]:
feature_set = train_df[['gre', 'gpa']] # Dont Need 2차원 Numpy 변환
feature_set.shape

(400, 2)

In [6]:
target_set = train_df['admit'].values.reshape(-1,1) # 2차원 Numpy 변환
target_set.shape

(400, 1)

####data Split
- train:test(7:3)

In [7]:
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(feature_set, target_set, test_size=0.3)

####Missing Value(결측치)

####이상치 처리 : 생략

####정규화(Normalization)

#####Min-Max Scaling

In [8]:
from sklearn.preprocessing import MinMaxScaler

#####feature data(train and test)

######Train data

In [9]:
feature_train_minmaxscaler = MinMaxScaler()
feature_train_minmaxscaler.fit(feature_train)
feature_train_minmaxscaler.scale_, feature_train_minmaxscaler.min_

(array([0.00172414, 0.63291139]), array([-0.37931034, -1.53164557]))

In [10]:
feature_train_normalization = feature_train_minmaxscaler.transform(feature_train)
feature_train_normalization.shape

(280, 2)

######Test data

In [11]:
feature_test_minmaxscaler = MinMaxScaler()
feature_test_minmaxscaler.fit(feature_test)
feature_test_minmaxscaler.scale_, feature_test_minmaxscaler.min_

(array([0.002     , 0.57471264]), array([-0.6       , -1.29885057]))

In [12]:
feature_test_normalization = feature_test_minmaxscaler.transform(feature_test)
feature_test_normalization.shape

(120, 2)

#####target data(train and test)

##### rename

In [13]:
feature_train = feature_train_normalization.copy()
feature_test = feature_test_normalization.copy()
#target_train = target_train_normalization.copy()
#target_test = target_test_normalization.copy()

###Model 구현

In [14]:
import tensorflow as tf

#### input layer

In [15]:
input = tf.keras.layers.Input(shape=(2,))   # feature column count

####hidden layers

In [16]:
hiddens = tf.keras.layers.Flatten()(input)

#### output layer

In [17]:
output = tf.keras.layers.Dense(units=1, activation='sigmoid')(hiddens)

#### Model 

In [18]:
model = tf.keras.Model(inputs=input, outputs=output)

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


###Model 설정

####Optimizer 선택

In [20]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

####Loss function 선택

In [21]:
#loss = tf.keras.losses.MeanSquaredError()
loss = tf.keras.losses.BinaryCrossentropy()

####Model 평가치 선택
- 데이터 형태 따라 평가치를 선택
- accuracy, precision, recall, fi score, AUC curve, logloss

In [22]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

###Model 학습

In [23]:
epoch = 1000
history = model.fit(x=feature_train, y=target_train, validation_split=0.3, epochs=epoch, shuffle=True, verbose=0)

###Model Evaluate

In [24]:
result_evaluation = model.evaluate(feature_test, target_test)
result_evaluation

4/4 [==============================] - 0s 4ms/step - loss: 0.6028 - accuracy: 0.7083


[0.6027643084526062, 0.7083333134651184]